In [1]:
import ast
import astor
import inspect

In [2]:
# convert string source code to ast
tree = ast.parse("def f(x): return 2 * (x ** 3) + 1", mode='exec')

In [3]:
# quick view of the entire ast
for statement in tree.body:
    print(ast.dump(statement), '\n')

FunctionDef(name='f', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=BinOp(left=Num(n=2), op=Mult(), right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=Num(n=3))), op=Add(), right=Num(n=1)))], decorator_list=[], returns=None) 



In [18]:
binop_stack = []
data_stack = []

# start from root
root = tree.body[0].body[0].value

def down(z):
    """
    using depth first search, add to stack if node is a binop otherwise
    evaluate leaf nodes. 
    
    all leaf nodes are evaluated, all non leaf nodes are binops to be 
    explored.
    """
    if isinstance(z, ast.BinOp):
        print("adding", z.op, "to binop stack")
        binop_stack.append(z)
        
        # a binop will always have a left and right child
        down(z.left)
        down(z.right)
        
        # once we've explore both left and right children of binop, we can remove it from stack
        binop_stack.pop()  
    else:
        evaluate(z)
        
def evaluate(z):
    """evaluate a leaf node.
    
    how a leaf is evaluated depends on its type (eg it is a ast.Name) 
    and parent's type (eg its parent is ast.Pow).
    """
    print('eval', z)
    parent = binop_stack[-1].op
    
    # scenario 1: node is a Name node
    if isinstance(z, ast.Name):
        if isinstance(parent, ast.Pow):
            exp = binop_stack[-1].right.n
            binop_stack[-1].right.n -= 1
            data_stack[-1].n = data_stack[-1].n * exp
            data_stack.pop()
    
    # scenario 2: node is a Num
    if isinstance(z, ast.Num):
        if isinstance(parent, ast.Mult):
            print("adding", z.n, "to data stack")
            data_stack.append(z)
        if isinstance(parent, ast.Add):
            print("stop")
            z.n = 0  # since any constants have zero derivative
        if isinstance(parent, ast.Pow):
            pass

In [14]:
down(root)

adding <_ast.Add object at 0x1044d8be0> to binop stack
adding <_ast.Mult object at 0x1044d8d30> to binop stack
eval <_ast.Num object at 0x106d58ac8>
adding 2 to data stack
adding <_ast.Pow object at 0x1044d8fd0> to binop stack
eval <_ast.Name object at 0x106d58b38>
eval <_ast.Num object at 0x106d58b70>
eval <_ast.Num object at 0x106d58ba8>
stop


In [15]:
curr = tree.body[0]  
print(astor.to_source(curr)) 

source = astor.to_source(curr)

def f(x):
    return 6 * x ** 2 + 0



In [16]:
# https://stackoverflow.com/questions/48759838/how-to-create-a-function-object-from-an-ast-functiondef-node
code = compile(tree, 'hello.py', 'exec')
namespace = {}
exec(code, namespace)  # put function code object into namespace dictionary

In [17]:
namespace["f"](1)

6